<a href="https://colab.research.google.com/github/wojiushilr/pytorch_training/blob/master/01_beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01_beginner
### Author: Lirui


In [2]:
import torch
import numpy as np

## 基础
### What is the tensor ?

In [29]:
# return 5*4 tensor
torch.Tensor(5, 4) #default tensor type (torch.FloatTensor).
# torch.tensor([2,1], dtype=torch.double)

tensor([2., 1.])

### 返回的数组大小是5x4的矩阵，初始化是0~1的均匀分布
### Construct a randomly initialized matrix:

In [9]:
torch.rand(5, 4)

tensor([[0.4469, 0.8075, 0.4858, 0.0165],
        [0.0073, 0.0184, 0.4979, 0.6757],
        [0.8141, 0.0524, 0.3400, 0.9390],
        [0.7108, 0.6492, 0.1922, 0.8952],
        [0.1420, 0.9768, 0.7989, 0.7161]])

### get the size

In [ ]:
torch.rand(5, 4).size()

### like np.ones()

In [3]:
# numpy 类似的返回5x4大小的矩阵
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


### Create a tensor based on an existing tensor. These methods will reuse properties of the input tensor
### e.g. dtype, unless new values are provided by user

In [ ]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)  

### numpy 和 torch.Tensor 之间的转换

In [ ]:
a = torch.rand(5, 4)
b = a.numpy()
print(b)
print(type(b))
a = np.array([i for i in range(10)])
b = torch.from_numpy(a)
print(b)
print(type(b))

### 运算和numpy类似

In [ ]:
x = torch.ones(3, 2)
y = torch.ones(3, 2)
c = 3
print(c * x)
print(x + y)
print(x.add(y))
print(x)
# 可以直接进行操作改变原对象，x+y或者x.add()并不会改变x，但是x.add_()则会对x进行改变
x.add_(y)
print(x)

# providing an output tensor as argument
result = torch.empty(3, 2)
torch.add(x, y, out=result)
print("result： ")
print(result)

### Resizing: If you want to resize/reshape tensor, you can use torch.view:

In [9]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


### 将 torch.Tensor 放到 GPU 上

In [36]:
# 判断一下电脑是否支持GPU
torch.cuda.is_available()

True

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[ 1.5585,  1.1210,  1.3479,  0.4689],
        [ 1.7637,  2.7982,  0.5568,  1.5845],
        [-0.4854, -0.4950,  0.0614,  0.9224],
        [ 1.7262,  0.7709,  2.1007, -0.0983]], device='cuda:0')
tensor([[ 1.5585,  1.1210,  1.3479,  0.4689],
        [ 1.7637,  2.7982,  0.5568,  1.5845],
        [-0.4854, -0.4950,  0.0614,  0.9224],
        [ 1.7262,  0.7709,  2.1007, -0.0983]], dtype=torch.float64)


## autograd


### torch 的自动求导功能
torch 和大部分框架一样有着自动求导功能，对象不再是 torch.Tensor，而是torch.autograd.Variable

本质上Variable和Tensor没有什么区别，不过Variable会放在一个计算图里面，可以进行前向传播和反向传播以及求导
[referance](https://pytorch.org/docs/stable/autograd.html)

In [76]:
# Create a tensor and set requires_grad=True to track computation with it
# set the object for autograd
x = torch.tensor([3], dtype = torch.float, requires_grad=True)
y = torch.ones(1, requires_grad=True)
print(x)
print(y)


tensor([3.], requires_grad=True)
tensor([1.], requires_grad=True)


In [77]:
# bulid the compute gragh
z = x * x + y
print(z)
print(z.grad_fn)

tensor([10.], grad_fn=<AddBackward0>)


In [78]:
out = z
print(out)


tensor([10.], grad_fn=<AddBackward0>)


### Gradients
### Let’s backprop now. Because out contains a single scalar, out.backward() is equivalent to out.backward(torch.tensor(1.)).

In [79]:
print(out)
## 对 x 和 y 分别求导
out.backward(retain_graph=True)

tensor([10.], grad_fn=<AddBackward0>)


### Print gradients d(out)/dx

In [81]:
# x 的导数和 y 的导数
print(x.grad)
print(y.grad)

tensor([6.])
tensor([1.])


### an example of vector-Jacobian product:


In [63]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ -126.7933,   605.1998, -1189.4474], grad_fn=<MulBackward0>)


In [64]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


### stop autograd from tracking history
- with torch.no_grad()
- or .detach()

In [ ]:
print((x ** 2).requires_grad)
with torch.no_grad():
    print((x ** 2).requires_grad)

y = x.detach()
print(y.requires_grad)
print(x.requires_grad)
print(x.eq(y).all())

### 改变requires_grad flag的值
### .requires_grad_( ... ) changes an existing Tensor’s requires_grad flag in-place. 

In [28]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True
